In [1]:

import csv
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from os import getcwd

In [2]:
def get_data(filename):
    with open(filename) as training_file:
        csv_reader=csv.reader(training_file,delimiter=',')
        first_line=True
        temp_labels=[]
        temp_images=[]
        for row in csv_reader:
            if first_line==True:
                first_line=False
            else:
                temp_labels.append(row[0])
                image_data=row[1:785]
                image_data_as_array=np.array_split(image_data,28)
                temp_images.append(image_data_as_array)
        images=np.array(temp_images).astype('float')
        labels=np.array(temp_labels).astype('float')
    return images, labels

path_sign_mnist_train = f"{getcwd()}/../tmp2/sign_mnist_train.csv"
path_sign_mnist_test = f"{getcwd()}/../tmp2/sign_mnist_test.csv"
training_images, training_labels = get_data(path_sign_mnist_train)
testing_images, testing_labels = get_data(path_sign_mnist_test)

print(training_images.shape)
print(training_labels.shape)
print(testing_images.shape)
print(testing_labels.shape)


(27455, 28, 28)
(27455,)
(7172, 28, 28)
(7172,)


In [3]:


training_images =np.expand_dims(training_images,axis=3)
testing_images = np.expand_dims(testing_images,axis=3)

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=0.2,
                                   fill_mode='nearest'
  
    )

validation_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=0.2,
                                   fill_mode='nearest'
               
    )
    
print(training_images.shape)
print(testing_images.shape)
    

(27455, 28, 28, 1)
(7172, 28, 28, 1)


In [6]:

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(26,activation='softmax')
])
from tensorflow.keras.optimizers import RMSprop 
model.compile(loss='sparse_categorical_crossentropy',optimizer=RMSprop(lr=0.001),metrics=['acc'])

history = model.fit_generator(# Your Code Here (set 'epochs' = 2)
                            train_datagen.flow(training_images, training_labels, batch_size=32),
                            steps_per_epoch=len(training_images)/32,
                            epochs=2,
                            validation_data=validation_datagen.flow(testing_images, testing_labels, batch_size=32),
                            validation_steps=len(testing_images)/32,
                        
                            )    

model.evaluate(testing_images, testing_labels, verbose=0)

Epoch 1/2
858/857 [==============================] - 83s 97ms/step - loss: 2.7609 - acc: 0.1667 - val_loss: 2.4752 - val_acc: 0.2322
Epoch 2/2
858/857 [==============================] - 82s 96ms/step - loss: 2.1876 - acc: 0.3174 - val_loss: 2.0499 - val_acc: 0.3345


[309.2819057619911, 0.23284997]

In [ ]:

%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()